In [1]:
import os
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from gym.envs.classic_control.cartpole import CartPoleEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [2]:
#env = gym.make("CartPole-v0")
env = CartPoleEnv()

### Understanding the enviroment

In [3]:
n_actions = env.action_space.n
n_obs = env.observation_space.shape[0]

for episode in range(10):
    obs = env.reset()
    done = 0
    score = 0

    while not done:
        action = env.action_space.sample()
        obs, reward, done, _ = env.step(action)
        env.render()
        score += reward
    
    print("Episode %d, Score %d"%(episode, score))

env.close() 


/usr/local/lib/python3.8/dist-packages/pyglet/window/__init__.py:667: UserWarning: 
Your graphics drivers do not support OpenGL 2.0.
You may experience rendering issues or crashes.
Intel
Intel(R) HD Graphics 620
1.4 (4.6.0 - Build 27.20.100.8681)
  warnings.warn(message)


Episode 0, Score 28
Episode 1, Score 35
Episode 2, Score 25
Episode 3, Score 22
Episode 4, Score 54
Episode 5, Score 12
Episode 6, Score 15
Episode 7, Score 13
Episode 8, Score 11
Episode 9, Score 19


### definig the Agent

In [3]:
# path to save the training logs and weidths 
log_path = os.path.join("Training", "Logs")
A2C_path = os.path.join("Training", "Saved_Models", "A2C")

### Defininf the callbaks


In [4]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=A2C_path, 
                             verbose=1)

In [5]:
env = DummyVecEnv([lambda: env])
model = A2C('MlpPolicy', env, tensorboard_log=log_path, seed=123, verbose=2)
#A2C?

Using cpu device


### training the agent

In [6]:
model.learn(total_timesteps=20000, tb_log_name='A2C_cartPole', log_interval=20, callback=eval_callback)
#model.learn

2021-12-13 03:57:13.934666: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2021-12-13 03:57:13.934725: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Logging to Training/Logs/A2C_cartPole_6
------------------------------------
| time/                 |          |
|    fps                | 36       |
|    iterations         | 20       |
|    time_elapsed       | 2        |
|    total_timesteps    | 100      |
| train/                |          |
|    entropy_loss       | -0.685   |
|    explained_variance | -0.00299 |
|    learning_rate      | 0.0007   |
|    n_updates          | 19       |
|    policy_loss        | 2.01     |
|    value_loss         | 10.4     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 67       |
|    iterations         | 40       |
|    time_elapsed       | 2        |
|    total_timesteps    | 200      |
| train/                |          |
|    entropy_loss       | -0.69    |
|    explained_variance | 0.163    |
|    learning_rate      | 0.0007   |
|    n_updates          | 39       |
|    policy_loss        | 1.65     

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=304.40 +/- 20.20
Episode length: 304.40 +/- 20.20
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 304      |
|    mean_reward        | 304      |
| time/                 |          |
|    total_timesteps    | 10000    |
| train/                |          |
|    entropy_loss       | -0.636   |
|    explained_variance | 4.49e-05 |
|    learning_rate      | 0.0007   |
|    n_updates          | 1999     |
|    policy_loss        | 0.226    |
|    value_loss         | 0.209    |
------------------------------------
New best mean reward!
Stopping training because the mean reward 304.40  is above the threshold 200


### Saving the model

In [10]:
model.save(A2C_path)

In [9]:
# evaluate policy
#evaluate_policy(model, env, n_eval_episodes=10, render=True, return_episode_rewards=True)
evaluate_policy?

Signature:
evaluate_policy(
    model: 'base_class.BaseAlgorithm',
    env: Union[gym.core.Env, stable_baselines3.common.vec_env.base_vec_env.VecEnv],
    n_eval_episodes: int = 10,
    deterministic: bool = True,
    render: bool = False,
    callback: Union[Callable[[Dict[str, Any], Dict[str, Any]], NoneType], NoneType] = None,
    reward_threshold: Union[float, NoneType] = None,
    return_episode_rewards: bool = False,
    warn: bool = True,
) -> Union[Tuple[float, float], Tuple[List[float], List[int]]]
Docstring:
Runs policy for ``n_eval_episodes`` episodes and returns average reward.
If a vector env is passed in, this divides the episodes to evaluate onto the
different elements of the vector env. This static division of work is done to
remove bias. See https://github.com/DLR-RM/stable-baselines3/issues/402 for more
details and discussion.

.. note::
    If environment has not been wrapped with ``Monitor`` wrapper, reward and
    episode lengths are counted as it appears with ``en

In [8]:
env.close()

### Testing the model 

In [13]:
del model
model = A2C.load(A2C_path)

In [9]:
obs = env.reset()
score= 0
while 1:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()
    score += reward
    if done:
        print("Score:", score)
        break
env.close()

/usr/local/lib/python3.8/dist-packages/pyglet/window/__init__.py:667: UserWarning: 
Your graphics drivers do not support OpenGL 2.0.
You may experience rendering issues or crashes.
Intel
Intel(R) HD Graphics 620
1.4 (4.6.0 - Build 27.20.100.8681)
  warnings.warn(message)


Score: [8.]
